## Load from PDF

In [ ]:
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Read the PDF and extract all text
pdf_reader = PdfReader(PDF_PATH)
print(f"📄 PDF loaded with {len(pdf_reader.pages)} pages")
# Extract text from all pages
raw_text = ""
for page_num, page in enumerate(pdf_reader.pages):
    page_text = page.extract_text()
    raw_text += page_text
print(f"📊 Extracted {len(raw_text)} characters total")

# Clean the extracted text
def clean_extracted_text(text: str) -> str:
    # Replace multiple whitespace with single spaces
    cleaned = re.sub(r'\s+', ' ', text)
    # Remove control characters
    cleaned = re.sub(r'[\x00-\x1F\x7F]', '', cleaned)
    # Strip leading/trailing whitespace
    return cleaned.strip()
document_text = clean_extracted_text(raw_text)
print(f"🧹 Cleaned text: {len(document_text)} characters")
print(f"📝 Preview: {document_text[:200]}...")

## Load From Web

In [2]:
! pip install bs4

  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Buddhist_ethics")
data = loader.load()

## Embed and Create DB

In [8]:
! pip install chromadb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.26.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached mdurl-0.1.2-py3-none-any.wh

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Better than CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings

# 1. Use RecursiveCharacterTextSplitter instead - handles long texts better
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # Try to split on paragraphs first
)

# 2. Clean your data first
def clean_documents(data):
    return [doc for doc in data if doc.page_content.strip()]  # Remove empty documents

cleaned_data = clean_documents(data)
docs = text_splitter.split_documents(cleaned_data)

# 3. Verify documents before embedding
print(f"Total documents after splitting: {len(docs)}")
for i, doc in enumerate(docs[:3]):  # Inspect first few
    print(f"Doc {i} length: {len(doc.page_content)}")
    print(f"Metadata: {doc.metadata}")

# 4. Create DB with error handling
try:
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma.from_documents(
        documents=docs,
        embedding=embedding_function,
        persist_directory='./ethics_db'
    )
    print("Database created successfully!")
except Exception as e:
    print(f"Error creating DB: {e}")
    # Inspect problematic documents
    for i, doc in enumerate(docs):
        if not doc.page_content or len(doc.page_content) > 2000:
            print(f"Problematic doc {i}: Length={len(doc.page_content)}")

Total documents after splitting: 160
Doc 0 length: 982
Metadata: {'source': 'https://en.wikipedia.org/wiki/Buddhist_ethics', 'title': 'Buddhist ethics - Wikipedia', 'language': 'en'}
Doc 1 length: 891
Metadata: {'source': 'https://en.wikipedia.org/wiki/Buddhist_ethics', 'title': 'Buddhist ethics - Wikipedia', 'language': 'en'}
Doc 2 length: 811
Metadata: {'source': 'https://en.wikipedia.org/wiki/Buddhist_ethics', 'title': 'Buddhist ethics - Wikipedia', 'language': 'en'}


/home/j/Documents/gyalpo/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Database created successfully!


/tmp/ipykernel_52237/466656509.py:34: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()
